# 1. Installing twint
Installing twint will install all related packages (like numpy,etc) that makes twint function properly.

### Upgrading twint
For those who already have twint and wish to upgrade it due to certain functionality not working, or any other reasons, run the uninstall command first.

Otherwise, for a fresh install, you may skip running the code below and start from the second cell

In [1]:

!pip3 uninstall twint --yes


Found existing installation: twint 2.1.20
Uninstalling twint-2.1.20:
  Successfully uninstalled twint-2.1.20


In [2]:
!pip3 install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master #egg=twint

  Cloning https://github.com/twintproject/twint.git (to revision origin/master) to /private/var/folders/1j/w13pmgjd6r59876tb75jxls80000gn/T/pip-req-build-_lb73rkj
  Running command git clone -q https://github.com/twintproject/twint.git /private/var/folders/1j/w13pmgjd6r59876tb75jxls80000gn/T/pip-req-build-_lb73rkj
  Running command git checkout -q origin/master
  Using cached dataclasses-0.6-py3-none-any.whl (14 kB)
  Created wheel for twint: filename=twint-2.1.21-py3-none-any.whl size=38760 sha256=e07ff87bfe6cad5c2b948cca583b2cd4bd3a38fc2bdd9d8b37661cb02d9eca27
  Stored in directory: /private/var/folders/1j/w13pmgjd6r59876tb75jxls80000gn/T/pip-ephem-wheel-cache-zjln_dgb/wheels/8d/dc/9f/74b4483d5f997036f04aec7f42bd4b3c80f04264920c368068
Successfully built twint
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/Users/pcowe/opt/anaconda3/bin/python -m pip install --upgrade pip' comma

In [2]:
!pip3 install nest_asyncio

You should consider upgrading via the '/Users/pcowe/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [13]:
!pip3 install python-dotenv

  Using cached python_dotenv-0.15.0-py2.py3-none-any.whl (18 kB)
You should consider upgrading via the '/Users/pcowe/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [1]:
!pip freeze > requirements.txt

# 2. Import Packages
Importing modules and essential packages for running this notebook :)

In [3]:
import os, sys, time, csv, math
# Basic utilities for processing data in python
import pandas as pd
import numpy as np
# Open source Twitter intelligence tool
import twint
# Twitter authentication client
from twitterclient import get_twitter_client
# Twitter API client
import tweepy
from tweepy import Cursor
# Removes runtimeError in jupyterlab -> This event loop is already running
import nest_asyncio
nest_asyncio.apply()

# 3. Find network actors using twint
Use the cell below to gather information/data about different search queries, hasthags, etc to identify which actors and conversations you wish to study.

All data from this section will be stored in '/search queries' folder

In [2]:
%%capture
# Configure twint query parameters to find all users talking about FSL

qp = twint.Config()
qp.Search = "French as a second language"
qp.Output = "data/search queries/fsl-test.csv"
qp.Store_csv = True

# Run twint on configured qp

twint.run.Search(qp)
print("Running search query")

## Other ways to find network actors (Domain knowledge)

A group of researchers with domain knowledge about FSL communities have shared some of their finidngs. I have discovered new users who are quite relevant to our analysis that I will manually add to our list

### Frequently mentioned users in posts discussing FSL (Research conducted by Liam Bekirsky)

In [4]:
liams_userlist = ['tdsb_fsl', 'tdsbvs', 'Natasha_Faroogh', 'scholasticCDA', 'OEEO_OCT', 'OCT_OEEO', 'sarnott_uottawa', 'EducLang', 'mimi_masson', 'MonsieurSteve1', 'CASLT_ACPLS', 'CPFontario', 'sdvlil', 'CecileRobertso5', 'Alex076', 'HDSBFSL', 'aliceF11', 'transformingfsl', 'ginagkozak1', 'edu_chantal', 'MmeCoulson', 'EricKeunne', 'FrenchStreetCa', 'CPFNational', 'stephendavisSK', 'mmecarr', 'Cinefranco', 'broadwayprofe', 'MlleMouland', 'booklamations', 'ClasseMHussain', 'SylviaJudek', 'MmeFagnan1', 'joel_055', 'RECRAE_RSEKN', 'KNAER_RSEKN', 'JoseeLeBouthill', 'cabirees', 'eostaffdevnet', 'DeniseAndre613']

# 4. Build network
In this step we need to identify all actors that we will include in the network we are going to study. To do this, we will extract usernames from all the user names we extracted in the /search queries folder 

In [5]:
# variables
MAX_FRIENDS = 15000
client = get_twitter_client()
max_pages = math.ceil(MAX_FRIENDS/ 5000)

In [6]:
#setup paginate function
def paginate(items, n):
    """
    Generate n-sized chunks from Items

    """
    for i in range(0, len(items), n):
        yield items[i:i+n]

In [7]:
#Open csv file to extract column names
with open('data/search queries/fsl-test.csv') as csv_file:
    csv_reader = csv.reader(csv_file)
    columnNames = []
    for row in csv_reader:
        columnNames.append(row)
        #break after reading first row
        break
columnNames = columnNames[0]


In [8]:
# Todo: Add code to extract list of username from multiple csv files

data = pd.read_csv('data/search queries/fsl-test.csv', names=columnNames)
usernames = data.username.tolist()

In [8]:
print(usernames[0])

1paulowe


## Username trimming
Because there is simply toooooo many tweets that meet our keyword criteria, we would like to intorduce an additional threshold value for us to consider a username. 

Most usernames, if not trimmed, are irrelevant to our search and may actually appear in our dataset only by chance and not because they regularly participate in the FSL discussion.

Our threshold cutoff would therefore be:  participations in the FSL discussion for us to consider expanding into any users network.

In [19]:
# How many users are in our list
print(len(usernames))

# Find duplicates
import collections
dupli = [item for item, count in collections.Counter(usernames).items() if count > 1]
#print(len(dupli))
#print(len(liams_userlist))

# For each duplicate find out how many times it actually occurs in our usernames list -  this will identify "active users in the FSL discussion"
shortlist = "data/usershortlist-original.csv"
fieldnames = ['Username', 'Count']      
with open(shortlist, 'a') as f:
    for u in dupli:
        csv_writer = csv.DictWriter(f, fieldnames=fieldnames)
        count = usernames.count(u)
        #print(u, count)
        csv_writer.writerow({'Username': u, 'Count': count})
        

### Semi-automatic trimming of shortlist dataset
Manually trimmed dataset of duplicates
- Kept usernames with 40+ occurences of the queried phrase
- Kept users who I recognized as part of the network (regardless of their threshold occurence value)

Next, read the semi-automated shortlist of users ....


In [23]:
# read shortlisted users

shortlist_attributes = ['Username', 'Count']
shortlist_file = pd.read_csv('data/usershortlist.csv', names=shortlist_attributes)
shortlisted = shortlist_file.Username.tolist()

In [30]:
# Append Liam's list and shortlisted list
usernames = []
print(len(shortlisted))
usernames.extend(shortlisted)
print(len(liams_userlist))
usernames.extend(liams_userlist)
print(len(usernames))

42
40
82


In [31]:
# Ensure you didnt introduce any further duplicates. We dont wanna do double scrape work :)
def checkIfDuplicates_1(listOfElems):
    ''' Check if given list contains any duplicates '''
    if len(listOfElems) == len(set(listOfElems)):
        return False
    else:
        return True
    
checkIfDuplicates_1(usernames)

False

## Creating Edges

In the next part I will create directed edges from our network of users. This is directed because we are goimg to represent followers/following relationships for each user in our general group of users.


In [33]:
# make sure you are not repeating already scraped users. You will waste time
# for this, check your network-progress list for users you already have scraped and remove them from usernames 

# load updated username list before you continue to build your network 

progress_params = ['explored_users', 'skipped_users']
network_progress = pd.read_csv('data/network/network-progress2.csv', names=progress_params)
explored_users = network_progress.explored_users.tolist()
skipped_users = network_progress.skipped_users.tolist()
# ignoring the nan's and column names, len(explored) + len(skipped) = len(usernames) 
unames = [ x for x in usernames if x not in (explored_users + skipped_users)]

#loop through all users and write their followers to csv
try:
    
    for user in unames:

        fname = "data/network/network-edges2.csv"
        fieldnames = ['Source', 'Target']
        with open(fname, 'a') as f:

            csv_writer = csv.DictWriter(f, fieldnames=fieldnames)

            # get a user's followers list
            for followers in Cursor(client.followers_ids, screen_name=user).pages(max_pages):
                for chunk in paginate(followers, 100):
                    userFollowers = client.lookup_users(user_ids=chunk)

                    for follower in userFollowers:

                        #print(len(users))

                        #print(follower.screen_name)
                        csv_writer.writerow({'Source': follower.screen_name, 'Target': user})

                    # f.write(json.dumps(user._json)+"\n")
                if len(followers) == 5000:
                    print("More results available. Sleeping for 60 seconds to avoid rate limit")
                    time.sleep(60)

            # get a user's following list
            for friends in Cursor(client.friends_ids, screen_name=user).pages(max_pages):

                for chunk in paginate(friends, 100):
                    userFollowing = client.lookup_users(user_ids=chunk)
                    for followed in userFollowing:
                        csv_writer.writerow({'Source': user, 'Target': followed.screen_name})
                if len(friends) == 5000:
                    print("More results available. Sleeping for 60 seconds to avoid rate limit")
                    time.sleep(60)

            f.close()
            
            # Update network-progress
            with open('data/network/network-progress2.csv', 'a') as nprog:
                nprog_updater = csv.DictWriter(nprog, fieldnames=progress_params)
                nprog_updater.writerow({'explored_users': user, 'skipped_users': ' '})
                nprog.close()
            
except tweepy.TweepError as err:
    
    
    print("Tweepy error occured: {}. Skipped adding user: {}".format(err, user))
    with open('data/network/network-progress2.csv', 'a') as nprog:
        nprog_updater = csv.DictWriter(nprog, fieldnames=progress_params)
        nprog_updater.writerow({'explored_users': ' ', 'skipped_users': user})
        nprog.close()
    pass
        
    
    

More results available. Sleeping for 60 seconds to avoid rate limit
More results available. Sleeping for 60 seconds to avoid rate limit
More results available. Sleeping for 60 seconds to avoid rate limit
More results available. Sleeping for 60 seconds to avoid rate limit
More results available. Sleeping for 60 seconds to avoid rate limit
More results available. Sleeping for 60 seconds to avoid rate limit
More results available. Sleeping for 60 seconds to avoid rate limit
More results available. Sleeping for 60 seconds to avoid rate limit
More results available. Sleeping for 60 seconds to avoid rate limit
More results available. Sleeping for 60 seconds to avoid rate limit
More results available. Sleeping for 60 seconds to avoid rate limit
More results available. Sleeping for 60 seconds to avoid rate limit
More results available. Sleeping for 60 seconds to avoid rate limit
More results available. Sleeping for 60 seconds to avoid rate limit
More results available. Sleeping for 60 seconds 

## Creating Nodes

In the next part I will create nodes from our network actors. 

### What I did
Used excel to find all unique names from my edge-list.
### Optional
You can add code instead find unique usernames and also you can include specific node attributes to enhance network visualization in gephi.


# 5. Send data to Gephi
In this step download gephi. It is an open source network analysis software

Next, watch a few youtube videos to understand how you can use gephi to upload your network nodes and edges.

Learn how to visualize your network on gephi 

![Network clusters](network-analysis/network-clusters.png)


Learn how to run algorithms such as PageRank, Eigenvector centrality and clustering algorithms on your network

Here's an example of community detection based on pagerank

![Network clusters](network-analysis/network-communitydetect.png)

# 6. Sentiment Analysis

# 7. Infographics